In [1]:
import numpy as np
import pandas as pd
import Quandl
import os
from sqlalchemy import sqlalchemy  # from flask.ext. 

Read Quandl data into a Pandas dataframe, tidy up the columns, verify data integrity and stash it in an SQL database.

In [2]:
ticker = "YAHOO/INDEX_GSPC"  # Set target ticker symbol
# In shell, prior to running notebook: export QUANDL_TOKEN="secret-key"
token = os.environ.get('QUANDL_TOKEN')  # Grab the environment variable

In [3]:
df = Quandl.get(ticker, authtoken=token, trim_start='1990-01-01')  # Using the Quandl module

In [4]:
df_devBackup = df.copy()  # For development only, copy the dataframe in case we make a mistake
# df = df_devBackup.copy()  # Restore the original dataframe

In [5]:
df  # Show dataframe

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
1990-01-02,353.399994,359.690002,351.980011,359.690002,162070000,359.690002
1990-01-03,359.690002,360.589996,357.890015,358.760010,192330000,358.760010
1990-01-04,358.760010,358.760010,352.890015,355.670013,177000000,355.670013
1990-01-05,355.670013,355.670013,351.350006,352.200012,158530000,352.200012
1990-01-08,352.200012,354.239990,350.540009,353.790009,140110000,353.790009
1990-01-09,353.829987,354.170013,349.609985,349.619995,155210000,349.619995
1990-01-10,349.619995,349.619995,344.320007,347.309998,175990000,347.309998
1990-01-11,347.309998,350.140015,347.309998,348.529999,154390000,348.529999
1990-01-12,348.529999,348.529999,339.489990,339.929993,183880000,339.929993


In [6]:
old_columns = list(df.columns.values)  # Get the column labels
ticker_tag = ticker.split('_')[-1] + '_'  # Use the ticker symbol as our new prefix
new_labels = [ticker_tag + i.replace(' ', '') for i in old_columns]  # Drop spaces and concatenate
new_columns = dict(zip(old_columns, new_labels))  # Create a dictionary of old and new column labels
new_columns  # Show the column label dictionary

{'Adj Close': 'GSPC_AdjClose',
 'Close': 'GSPC_Close',
 'High': 'GSPC_High',
 'Low': 'GSPC_Low',
 'Open': 'GSPC_Open',
 'Volume': 'GSPC_Volume'}

In [7]:
df = df.rename(columns=new_columns)  # Rename the columns using our dictionary

In [8]:
df.tail()  # Show tail-end of dataframe

,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Volume,GSPC_AdjClose
Date,,,,,,
2016-01-13,1940.339966,1950.329956,1886.410034,1890.280029,5087030000,1890.280029
2016-01-14,1891.680054,1934.469971,1878.930054,1921.839966,5241110000,1921.839966
2016-01-15,1916.680054,1916.680054,1857.829956,1880.329956,5468460000,1880.329956
2016-01-19,1888.660034,1901.439941,1864.599976,1881.329956,4928350000,1881.329956
2016-01-20,1876.180054,1876.180054,1812.290039,1859.329956,6416070000,1859.329956


In [9]:
nulls = df[~df.applymap(np.isreal).all(1)]  # Search for non-real numbers by negation
nulls  # Show any rows in the dataframe with non-numeric values 

,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Volume,GSPC_AdjClose
Date,,,,,,


In [10]:
# Add error checking for non-numeric values
if len(nulls) > 0:
    raise ValueError('Dataframe contains non-numeric values')

In [12]:
# Backup data to sql
engine = sqlalchemy.create_engine('sqlite:///data/dev.db')  
df.to_sql('gspc', engine, if_exists='replace')

df_test = pd.read_sql('gspc', engine)
df_test = df_test.set_index('Date')
df_test

,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Volume,GSPC_AdjClose
Date,,,,,,
1990-01-02,353.399994,359.690002,351.980011,359.690002,162070000,359.690002
1990-01-03,359.690002,360.589996,357.890015,358.760010,192330000,358.760010
1990-01-04,358.760010,358.760010,352.890015,355.670013,177000000,355.670013
1990-01-05,355.670013,355.670013,351.350006,352.200012,158530000,352.200012
1990-01-08,352.200012,354.239990,350.540009,353.790009,140110000,353.790009
1990-01-09,353.829987,354.170013,349.609985,349.619995,155210000,349.619995
1990-01-10,349.619995,349.619995,344.320007,347.309998,175990000,347.309998
1990-01-11,347.309998,350.140015,347.309998,348.529999,154390000,348.529999
1990-01-12,348.529999,348.529999,339.489990,339.929993,183880000,339.929993


In [13]:
# Backup data to csv
df.to_csv('data/gspc.csv')

df_test = pd.read_csv('data/gspc.csv')
df_test = df_test.set_index('Date')
df_test

,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Volume,GSPC_AdjClose
Date,,,,,,
1990-01-02,353.399994,359.690002,351.980011,359.690002,162070000,359.690002
1990-01-03,359.690002,360.589996,357.890015,358.760010,192330000,358.760010
1990-01-04,358.760010,358.760010,352.890015,355.670013,177000000,355.670013
1990-01-05,355.670013,355.670013,351.350006,352.200012,158530000,352.200012
1990-01-08,352.200012,354.239990,350.540009,353.790009,140110000,353.790009
1990-01-09,353.829987,354.170013,349.609985,349.619995,155210000,349.619995
1990-01-10,349.619995,349.619995,344.320007,347.309998,175990000,347.309998
1990-01-11,347.309998,350.140015,347.309998,348.529999,154390000,348.529999
1990-01-12,348.529999,348.529999,339.489990,339.929993,183880000,339.929993


In [14]:
def load(ticker):
    """Load data from Quandl into a dataframe, modify 
    column names and check for non-numeric values."""
    # Grab the Quandl token
    token = os.environ.get('QUANDL_TOKEN')
    if token is None:
        token = input("Enter Quandl token: ")
    # Call Quandl module, trim input by default from 1990 forward
    df = Quandl.get(ticker, authtoken=token, trim_start='1990-01-01')  
    # Get the column labels
    old_columns = list(df.columns.values)  
    # Use the ticker symbol as our new prefix
    ticker_tag = ticker.split('_')[-1] + '_'  
    # Drop spaces and concatenate
    new_labels = [ticker_tag + i.replace(' ', '') for i in old_columns]  
    # Create a dictionary of old and new column labels
    new_columns = dict(zip(old_columns, new_labels))  
    # Rename the columns using our dictionary
    df = df.rename(columns=new_columns)  
    nulls = df[~df.applymap(np.isreal).all(1)]
    # Check for non-numeric values
    if len(nulls) > 0:
        raise ValueError('Dataframe contains non-numeric values')
    row_count = len(df)
    print('%d rows loaded into dataframe.' % row_count) 
    return df

In [15]:
df = load("YAHOO/INDEX_GSPC")

6565 rows loaded into dataframe.
